In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import time
from scipy.io import wavfile
from scipy import signal
import cv2
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import time

In [3]:
#считывание файла
df = pd.read_csv(r'C:\Users\skills\Downloads\Модуль 3\training_datasets\train_shapes.csv.')
df

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,5/31/2005 01:20,doncaster (south yorkshire) (uk/england),NaN,gb,unknown,600,10 minutes,Bright red light hovering and taking off,6/20/2005,53.533333,-1.116667
1,11/10/2002 05:32,southfield,mi,us,other,5,5 secs? more or less,I HAVE NEVER HEARD A SOUND LIKE THIS COMING FR...,11/16/2002,42.473333,-83.221944
2,12/25/2009 00:30,seabrook,tx,us,unknown,5,5 seconds,A rather large yellow laser beam tracked from ...,2/14/2010,29.563889,-95.025278
3,12/16/1985 19:00,vallejo,ca,us,triangle,0,30,A shaped object with amber/organge lights&#44 ...,10/30/2006,38.104167,-122.255556
4,9/30/2005 18:28,coral springs,fl,us,disk,60.0,1 minute,SHIP BY THE SUN TOOK A PICTURE IN AMAZEMENT,10/11/2005,26.270833,-80.270833
...,...,...,...,...,...,...,...,...,...,...,...
62070,11/14/1997 21:10,snohomish,wa,us,formation,10,10 seconds,Large object seen traveling northeast at a slo...,1/28/1999,47.913056,-122.096944
62071,9/30/2000 20:00,globe,az,us,circle,30.0,15-30 sec.,Object was brillant green &#44marble shaped &...,12/2/2000,33.394167,-110.785833
62072,11/24/2013 18:26,lake stevens,wa,us,changing,30,30 seconds,Odd multishaped Orange Orbs Seen Over Western ...,12/2/2013,48.015278,-122.062500
62073,3/18/2014 03:00,berwyn,il,us,light,15,10-15 seconds,BRIGHT RED/ORANGE LIKE LIGHT CONSUMED ENTIRE L...,3/18/2014,41.850556,-87.793611


In [4]:
# копия для проверки Nan значений
df_copy = df.dropna()
df_copy

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
1,11/10/2002 05:32,southfield,mi,us,other,5,5 secs? more or less,I HAVE NEVER HEARD A SOUND LIKE THIS COMING FR...,11/16/2002,42.473333,-83.221944
2,12/25/2009 00:30,seabrook,tx,us,unknown,5,5 seconds,A rather large yellow laser beam tracked from ...,2/14/2010,29.563889,-95.025278
3,12/16/1985 19:00,vallejo,ca,us,triangle,0,30,A shaped object with amber/organge lights&#44 ...,10/30/2006,38.104167,-122.255556
4,9/30/2005 18:28,coral springs,fl,us,disk,60.0,1 minute,SHIP BY THE SUN TOOK A PICTURE IN AMAZEMENT,10/11/2005,26.270833,-80.270833
5,10/29/2011 23:00,foxborough,ma,us,flash,2,seconds,Unidentified Green flashing Light During Blizzard,12/12/2011,42.065278,-71.248333
...,...,...,...,...,...,...,...,...,...,...,...
62070,11/14/1997 21:10,snohomish,wa,us,formation,10,10 seconds,Large object seen traveling northeast at a slo...,1/28/1999,47.913056,-122.096944
62071,9/30/2000 20:00,globe,az,us,circle,30.0,15-30 sec.,Object was brillant green &#44marble shaped &...,12/2/2000,33.394167,-110.785833
62072,11/24/2013 18:26,lake stevens,wa,us,changing,30,30 seconds,Odd multishaped Orange Orbs Seen Over Western ...,12/2/2013,48.015278,-122.062500
62073,3/18/2014 03:00,berwyn,il,us,light,15,10-15 seconds,BRIGHT RED/ORANGE LIKE LIGHT CONSUMED ENTIRE L...,3/18/2014,41.850556,-87.793611
